# Multi-swarm Test

In [ ]:
# Swarm config
# num_swarm = 2
num_swarm = 4

# Agent config
num_agents = 50
agent_radius = 1

# Instance config, ratio of starts and goals
starts_weights = [[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]]
goals_weights = [[0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0]]
# starts_weights = [[1, 0, 0, 0], [0, 0, 1, 0]]
# goals_weights = [[0, 0, 1, 0], [1, 0, 0, 0]]

# starts_weight_2 = [0.5, 0.5]
# goals_weight_2 = [0.5, 0.5]
# starts_weight_1 = [0.5, 0.5]
# goals_weight_1 = [0.5, 0.5]


In [ ]:
# Load map and instance
import os
import pickle

from swarm_prm.utils.agent_assignment import get_agent_assignment

fname = "solutions/gaussian_prm.pkl"
with open(fname, "rb") as f:
    gaussian_prm = pickle.load(f)

starts_agent_counts = []
goals_agent_counts = []
for starts_weight, goals_weight in zip(starts_weights, goals_weights):
    starts_agent_counts.append(get_agent_assignment(num_agents, starts_weight))
    goals_agent_counts.append(get_agent_assignment(num_agents, goals_weight))

In [ ]:
# TEG
from swarm_prm.solvers.macro.teg import TEG

flow_dicts = []
capacity_dicts = []
for swarm in range(num_swarm):
    max_timestep = 0
    teg = TEG(gaussian_prm, agent_radius, 
                  starts_agent_count=starts_agent_counts[swarm], 
                  goals_agent_count=goals_agent_counts[swarm],
                  num_agents=num_agents, 
                  flow_dicts=flow_dicts,
                  capacity_dicts=capacity_dicts,
                  timestep=max_timestep, 
                  time_limit=10)

    timestep, flow_dict, capacity_dict= teg.get_solution()
    max_timestep = max(max_timestep, timestep)
    flow_dicts.append(flow_dict)
    capacity_dicts.append(capacity_dict)

In [ ]:
from matplotlib import pyplot as plt
from swarm_prm.solvers.micro import GaussianTrajectorySolver

simple_paths = []
gt_paths = []
for swarm in range(num_swarm):
    simple_path, _ = gaussian_prm.get_solution(flow_dicts[swarm], timestep, num_agents)
    macro_solution = gaussian_prm.get_macro_solution(flow_dicts[swarm])
    timestep = max(macro_solution.keys())

    gt_solver = GaussianTrajectorySolver(gaussian_prm, macro_solution, timestep, num_agents, 
                                         starts_agent_count=starts_agent_counts[swarm],
                                         goals_agent_count=goals_agent_counts[swarm],
                                         safety_gap=0)
    gt_path = gt_solver.solve()

    simple_paths.append(simple_path)
    gt_paths.append(gt_path)

fig, ax = gaussian_prm.visualize_g_nodes()

for simple_path in simple_paths:
    for path in simple_path:
        x_coords = [loc[0] for loc in path]
        y_coords = [loc[1] for loc in path]
        ax.plot(x_coords, y_coords, '-*', label='Path', color='blue')
# 
cmap = plt.get_cmap("tab10")

for i, swarm in enumerate(range(num_swarm)):
    for j, path in enumerate(gt_paths[swarm]):
        x_coords = [loc[0] for loc in path]
        y_coords = [loc[1] for loc in path]
        ax.plot(x_coords, y_coords, '--', label='Path', color=cmap(i))
plt.show()

# TODO: normalize solution length

In [ ]:
# Add interpolations 

import numpy as np

def add_linear_interpolation_points(path, subdiv=5):
    """
    Insert linearly-interpolated positions between consecutive points.
    
    Parameters
    ----------
    path : array-like of shape (N, 2)
        The original list/array of N 2D points, e.g. [(x1,y1), (x2,y2), ...].
    subdiv : int
        Number of intervals to subdivide each segment. 
        For each original segment, we will add 'subdiv - 1' new interior points 
        (except that we skip duplicates at the shared boundary).

    Returns
    -------
    new_path : np.ndarray of shape (M, 2)
        The new path including the original points and the added interpolation points.
    """
    path = np.asarray(path)
    new_path = []

    # Iterate over pairs of consecutive points
    for i in range(len(path) - 1):
        start = path[i]
        end = path[i + 1]
        
        # Create subdiv+1 points in [start, end] using linspace for each dimension
        xs = np.linspace(start[0], end[0], subdiv + 1)
        ys = np.linspace(start[1], end[1], subdiv + 1)
        
        # Add each intermediate point except the very last one
        # to avoid duplicating the next segment's start
        for j in range(subdiv):
            new_path.append([xs[j], ys[j]])

    # Finally add the very last point of the last segment
    new_path.append(path[-1].tolist())
    
    return np.array(new_path)

interpolated_paths = []
for gt_path in gt_paths:
    for path in gt_path:
        new_path = add_linear_interpolation_points(path, 50)
        interpolated_paths.append(new_path)

# add padding for trajectory
max_path_length = 0
for path in interpolated_paths:
    max_path_length = max(max_path_length, len(path))

padded_paths  = []
for path in interpolated_paths:
    pad = np.tile(path[-1, :], (max_path_length - len(path), 1))

    padded_path = np.concat((path, pad))
    padded_paths.append(padded_path)


In [ ]:
# Animate solution
from matplotlib.patches import Circle
from matplotlib.animation import FuncAnimation

def animate_solution(fig, ax, speed, paths):
    """
        Visualize solution trajectory provided instance
    """
    
    agents = []
    def init():
        for agent in agents:
            agent.remove()
        agents.clear()
        return []

    def update(frame):
        idx = frame * speed
        for agent in agents:
            agent.remove()
        agents.clear()
        cmap = plt.get_cmap("tab10")
        locs = [path[idx] for path in paths]
        for i, loc in enumerate(locs):
            agent = ax.add_patch(Circle(loc, radius=agent_radius, color=cmap(i%10)))
            agents.append(agent)
        return agents

    anim = FuncAnimation(fig, update, frames=len(paths[0])// speed , 
                         init_func=init, blit=True, interval=10)
    anim.save("solutions/apf_solution.gif", writer='pillow', fps=6)

# fig, ax = instance.visualize()
fig, ax = gaussian_prm.visualize_g_nodes()
animate_solution(fig, ax, 10, padded_paths)